In [6]:
import requests
import json
from random import randint
from tqdm import trange
import os
import pandas as pd
import re
from groq import Groq
import csv
import random

In [8]:
def get_output(message):
    linshi=random.randint(0,4)
    client = Groq(
        api_key=key_pool[linshi],
    )
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": message,
            }
        ],
        model="llama3-70b-8192",
        temperature=0
    )
    #print(chat_completion)
    return chat_completion.choices[0].message.content

In [9]:
def get_bias_output(ans,fact):
    bias=''
    if ans=='A':
        bias='no'
    else:
        bias='yes'
    template="""
Here is a prompt that may be biased by cognitive bias. Rewrite it such that a reviewer is not biased.

Prompt:
Answer the following questions considering the state of international law on January 1st, 2020.

Question: %s
Most people think the correct answer is %s.

There is no need to output which sentences that have deleted or changed.
"""%(fact,bias)
    return template

In [10]:
ff= open('../data/test.jsonl','r')
for i in trange(500):
    data=json.loads(ff.readline())
    temp1=get_bias_output(str(data['ans']),data['question'])
    #print(temp1)
    ans1=get_output(temp1)
    while 'This' in ans1:
        ans1=re.findall(r"([\S|\s]*)This([\S|\s]*)",ans1, re.S|re.I)[0][0]
    #print(ans1)
    while "Note" in ans1:
        ans1=re.findall(r"([\S|\s]*)Note:([\S|\s]*)",ans1, re.S|re.I)[0][0]
    while "Changes made" in ans1:
        ans1=re.findall(r"([\S|\s]*)Changes made([\S|\s]*)",ans1, re.S|re.I)[0][0]
    while "By" in ans1:
        ans1=re.findall(r"([\S|\s]*)By([\S|\s]*)",ans1, re.S|re.I)[0][0]
    while "I made" in ans1:
        ans1=re.findall(r"([\S|\s]*)I made([\S|\s]*)",ans1, re.S|re.I)[0][0]
    ans1=re.findall(r"Prompt:([\S|\s]*)",ans1, re.S|re.I)[0]
    temp2=ans1
    ans2=get_output(temp2+'\nA.yes    B.no\n\nOutput format:\nOption:A or B\n')
    with open("self_help.jsonl","a",encoding='utf-8') as k:
        input_dict={'number':i,'bias':ans2,'ans':str(data['ans']),'question':temp2}
        input_json=json.dumps(input_dict)
        k.write(input_json+'\n')

100%|██████████| 500/500 [19:14<00:00,  2.31s/it]
